In [ ]:
import os
import pandas as pd
import numpy as np
import gc
import json
#import joblib
from scipy.stats import pearsonr
from matplotlib import pyplot
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from itertools import compress
#import xgboost as xgb
#import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFECV
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
#from xgboost import XGBClassifier
#from xgboost import plot_importance
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
#import pydotplus
from sklearn import tree
from IPython.display import Image  
#import shap
import math
#shap.initjs()
pd.set_option('display.max_rows',100)
pd.set_option('display.max_columns',100)
gc.enable()

%matplotlib inline


In [ ]:
data=pd.read_csv('LoanStats3a.csv',skip_blank_lines = True,skiprows = 1,skipfooter = 2,engine = 'python')

In [ ]:
data.head()

In [ ]:
# check null percentage for all columns
null_df=data.isnull().mean().reset_index()
null_df.columns=['column_name','null_percent']
null_df.sort_values(by='null_percent',ascending=False, inplace=True)

null_df.plot(x='column_name',y='null_percent', style='o')

# Data Cleaning and Feature Engineerig

In [ ]:
# drop columns with NA percentage>0.99
null_cols=null_df[null_df['null_percent']>0.99].column_name.tolist()
null_cols
df0=data.drop(null_cols,axis=1)
print("Dropping null columns...")

In [ ]:
#Drop rows with NA in target variable
df0.dropna(subset=['loan_status'],inplace=True)

In [ ]:
#check loan_status counts for each category
df0['loan_status'].value_counts()

In [ ]:
df0.loan_status.unique()

In [ ]:
# Derive response variable "default"
df0['default'] = df0.loan_status.apply(lambda x: 1 if x == "Charged Off" or x == "Does not meet the credit policy. Status:Charged Off" else 0)

In [ ]:
df0.describe()

In [ ]:
# transform data types
# create a new feature 'credit_len'
# drop columns ['issue_d','earliest_cr_line']
df0['credit_len'] = (pd.to_datetime(df0['issue_d']) - pd.to_datetime(df0['earliest_cr_line'])).dt.days
df0.drop(['issue_d','earliest_cr_line'],axis = 1, inplace = True)

In [ ]:
# fill NA in credit_len with 0
df0['credit_len'].fillna(0, inplace=True)
df0['credit_len'].head()

In [ ]:
# drop columns that happen after default
drop_attr=['title','out_prncp','out_prncp_inv',
           'policy_code', 'delinq_amnt', 
           'recoveries','total_rec_late_fee',
           'last_pymnt_amnt','application_type','desc',
            'emp_title','loan_status',
             'zip_code','addr_state','total_rec_prncp',
            'total_rec_int','total_pymnt',
            'pymnt_plan','initial_list_status', 'last_pymnt_d',
            'last_credit_pull_d','hardship_flag','tax_liens',
            'total_pymnt_inv','collection_recovery_fee','debt_settlement_flag','next_pymnt_d','pymnt_plan']
            

In [ ]:
#drop variables that are not available at the time of application
df0.drop(drop_attr,inplace=True, axis=1)
df0.shape

In [ ]:
#check the only one factor columns
columns=df0.columns[df0.nunique(dropna = False)==1]
columns

In [ ]:
#drop one factor columns
df0.drop(columns=df0.columns[df0.nunique(dropna = False)==1], inplace=True)
df0.shape

In [ ]:
#transform 'int_rate' and 'revol_util' to numerical variable
df0['int_rate'] = df0.int_rate.str.extract('(\d+)').astype('float')
df0['revol_util'] = df0.revol_util.str.extract('(\d+)').astype('float')

In [ ]:
#transform emp_length to numerical variable
mapping_dict = {
"emp_length": {
"10+ years": 10,
"9 years": 9,
"8 years": 8,
"7 years": 7,
"6 years": 6,
"5 years": 5,
"4 years": 4,
"3 years": 3,
"2 years": 2,
"1 year": 1,
"< 1 year": 0,
"n/a": 0
},
"grade":{
"A": 1,
"B": 2,
"C": 3,
"D": 4,
"E": 5,
"F": 6,
"G": 7
}
}
df0 = df0.replace(mapping_dict)
df0[['emp_length','grade']].head()

In [ ]:
#categorical variables
attr_cat=['term', 'home_ownership','verification_status','purpose']
attr_o = [col for col in df0.select_dtypes(include = 'object').columns.tolist() if col not in ['grade','sub_grade']]

In [ ]:
# dummy categorical variables
df1 = pd.get_dummies(df0,prefix_sep = '_', columns = attr_o, dummy_na = True, sparse = False)

In [ ]:
df1['pub_rec_bankruptcies'].isna().sum()

In [ ]:
attr_mis=['mths_since_last_delinq','revol_util','collections_12_mths_ex_med',
          'chargeoff_within_12_mths','pub_rec_bankruptcies']

In [ ]:
df2=df1[np.isfinite(df1['delinq_2yrs'])]
df3=df2[np.isfinite(df2['credit_len'])]
df3.isnull().sum()[df3.isnull().sum()>0]

In [ ]:
# create new columns with columnname_nan
df4=df3[attr_mis].isnull().astype(int).add_suffix('_nan')
df5=pd.concat([df3,df4],axis=1)
df5.head()

In [ ]:
#fill na with 0
#df5.fillna(df5.median(skipna=True),inplace=True)
df5.fillna(0, inplace=True)

# Build Logistic Model

In [ ]:
#features
attr = [col for col in df5.columns if col not in ['default','sub_grade','total_pymnt','total_pymnt_inv']]

In [ ]:
# train, test datasets
train_x,test_x,train_y,test_y=train_test_split(df5[attr],df5['default'],test_size=0.3,random_state=123)

In [ ]:
#Logistic Regression Model
test=pd.concat([train_x,train_y],axis=1)
test[test.columns].corr()['default']

selector=RFECV(estimator=LogisticRegression(),step=1,cv=5,scoring='roc_auc')
selector.fit(train_x,train_y)
#plt.figure()
#plt.plot(range(1,len(selector.grid_scores_)+1),selector.grid_scores_)
#plt.show()
selector.support_
selector.ranking_
selector.n_features_
features=train_x.columns[selector.support_]
train_x_1=train_x[features]
test_x_1=test_x[features]

cor=train_x_1.corr()
mask=np.zeros_like(cor,dtype=np.bool)
sns.heatmap(cor,mask=mask,center=0,square=True,linewidths=0.5)
plt.show()


In [ ]:
#Use lasso to select features
param_grid = [
    {
     'penalty' : ['l1', 'l2'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['liblinear']},
]
clf = GridSearchCV(estimator=LogisticRegression(), param_grid = param_grid, scoring='roc_auc',cv = 5, verbose=True, n_jobs=-1)
clf.fit(train_x_1,train_y)
print(clf.best_params_)

lr=LogisticRegression(C=1.62,penalty='l1', solver='liblinear')
lr.fit(train_x_1,train_y)

feature_df=pd.DataFrame({'feature':train_x_1.columns.get_values(),'coef':lr.coef_[0]})
feature_l=train_x_1.columns[lr.coef_[0]!=0]
pred_lr_train=lr.predict_proba(train_x_1)[:,1]
roc_auc_score(train_y,pred_lr_train)
#0.7007666596172346
pred_lr_test=lr.predict_proba(test_x_1)[:,1]
roc_auc_score(test_y,pred_lr_test)
#0.6969112243485446

In [ ]:
roc_auc_score(train_y,pred_lr_train)

In [ ]:
roc_auc_score(test_y,pred_lr_test)

# Data exploration

In [ ]:
df=data.copy()

In [ ]:
df['issue_d'] = pd.to_datetime(df['issue_d'])

In [ ]:
df['year']=df['issue_d'].dt.year.astype(int,errors='ignore')
df['default'] = df.loan_status.apply(lambda x: 1 if x == "Charged Off" or x == "Does not meet the credit policy. Status:Charged Off" else 0)

In [ ]:
df['year'].head()

In [ ]:
# The year of 2011 was the year with the highest amount of loans were issued 
# Loans increase gradually by year from 2007 to 2011.
plt.figure(figsize=(12,8))
#sns.barplot('year', 'loan_amnt', data=df, palette='tab10')
df.groupby('year')['loan_amnt'].sum().plot(kind='bar')
plt.title('Issuance of Loans', fontsize=20)
plt.xlabel('Year', fontsize=20)
plt.ylabel('Total loan amount issued', fontsize=20)

In [ ]:
f, ax = plt.subplots(1,2, figsize=(16,8))

colors = ["#3791D7", "#D72626"]
labels ="Good Loans", "Bad Loans"

plt.suptitle('Information on Loan Conditions', fontsize=20)

df0["default"].value_counts().plot.pie(explode=[0,0.25], autopct='%1.2f%%', ax=ax[0], shadow=True, colors=colors, 
                                             labels=labels, fontsize=12, startangle=70)


# ax[0].set_title('State of Loan', fontsize=16)
ax[0].set_ylabel('% of Condition of Loans', fontsize=14)

# sns.countplot('loan_condition', data=df, ax=ax[1], palette=colors)
# ax[1].set_title('Condition of Loans', fontsize=20)
# ax[1].set_xticklabels(['Good', 'Bad'], rotation='horizontal')
palette = ["#3791D7", "#E01E1B"]

sns.barplot(x="year", y="loan_amnt", hue="default", data=df, palette=palette, estimator=lambda x: len(x) / len(df) * 100)
ax[1].set(ylabel="(%)")


In [ ]:
df0['int_rate'].describe()

In [ ]:
# Average interest is 11.59% Anything above this will be considered of high risk let's see if this is true.
df0['interest_level'] = np.nan
lst = [df0]

for col in lst:
    col.loc[col['int_rate'] <= 13.23, 'interest_level'] = 'Low'
    col.loc[col['int_rate'] > 13.23, 'interest_level'] = 'High'

In [ ]:
from scipy.stats import norm

plt.figure(figsize=(20,10))

palette = ['#009393', '#930000']
plt.subplot(221)
ax = sns.countplot(x='interest_level', data=df0, 
                  palette=palette, hue='default')

ax.set_title('The impact of interest rate \n on the condition of the loan', fontsize=14)
ax.set_xlabel('Level of Interest Payments', fontsize=12)
ax.set_ylabel('Count')

plt.subplot(222)
#ax1 = sns.countplot(x='interest_level', data=df0, 
                   #palette=palette, hue='term')
ax1 = sns.countplot(x='term', data=df0, 
                   palette=palette, hue='default')
ax1.set_title('The impact of maturity date \n on the condition of the loan', fontsize=14)
ax1.set_xlabel('Term', fontsize=12)
ax1.set_ylabel('Count')


plt.show()

In [ ]:
df.columns.tolist()

In [ ]:
# Total 
total_funded_amount=df['funded_amnt_inv'].sum()
total_payment_amount=df['total_pymnt_inv'].sum()

total_return_rate=(total_payment_amount-total_funded_amount)/total_funded_amount
total_return_rate

In [ ]:
d1=df.groupby('year')['funded_amnt_inv','total_pymnt_inv'].sum().reset_index().sort_values(by='year')
d1['return_rate']= (d1['total_pymnt_inv']-d1['funded_amnt_inv'])/d1['funded_amnt_inv']
d1

In [ ]:
# return rate

plt.clf()
d1.groupby('year')['return_rate'].sum().plot(kind='bar')
plt.show()